In [ ]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime


model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

model_saved=model.fit(
        training_set,
        epochs=10,
        validation_data=test_set,

        )

model.save('mymodel.h5',model_saved) ##hello 

#To test for individual images

mymodel=load_model('mymodel.h5')
test_image=image.load_img(r"C:\Users\Rajvardhan\OneDrive\Desktop\FaceMaskDetector\test\without_mask\30.jpg",target_size=(150,150,3))
test_image=image.load_img(r"C:\Users\Rajvardhan\OneDrive\Desktop\FaceMaskDetector\test\with_mask\1-with-mask.jpg",
                          target_size=(150,150,3))
test_image
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
mymodel.predict(test_image)[0][0]


# IMPLEMENTING LIVE DETECTION OF FACE MASK

mymodel=load_model('mymodel.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()


Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.
Epoch 1/10
83/83 [==============================] - 37s 412ms/step - loss: 0.3425 - accuracy: 0.8311 - val_loss: 0.1091 - val_accuracy: 0.9485
Epoch 2/10
83/83 [==============================] - 33s 392ms/step - loss: 0.1308 - accuracy: 0.9566 - val_loss: 0.1329 - val_accuracy: 0.9485
Epoch 3/10
83/83 [==============================] - 33s 395ms/step - loss: 0.1145 - accuracy: 0.9536 - val_loss: 0.0768 - val_accuracy: 0.9794
Epoch 4/10
83/83 [==============================] - 33s 391ms/step - loss: 0.0873 - accuracy: 0.9696 - val_loss: 0.1475 - val_accuracy: 0.9485
Epoch 5/10
83/83 [==============================] - 32s 389ms/step - loss: 0.0894 - accuracy: 0.9680 - val_loss: 0.0563 - val_accuracy: 0.9845
Epoch 6/10
83/83 [==============================] - 32s 389ms/step - loss: 0.1117 - accuracy: 0.9612 - val_loss: 0.0807 - val_accuracy: 0.9639
Epoch 7/10
83/83 [==============================] - 33s 393

C:\Users\Rajvardhan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - ETA: 0s